In [30]:
# -*- coding: utf-8 -*-

import numpy as np # biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd # biblioteca usada para trabalhar com dataframes e análise de dados
import sklearn as skl # sckit-learn
import time # biblioteca usada para calcular o tempo de execucao
import re # biblioteca para expressoes regulares
import os # biblioteca usada para realizar tarefas específicas ao SO


# importa alguns pacotes do sckit-learn
from sklearn import model_selection 
from sklearn import naive_bayes # necessario para usar o metodo naive Bayes
from sklearn import model_selection # necessario para fazer validacao cruzada
from sklearn import metrics # necessario para obter o desempenho da classificacao

import nltk # biblioteca para Processamento de Linguagem Natural
from nltk.stem.porter import PorterStemmer # para fazer a estemização em documentos da lingua inglesa
from nltk.stem import RSLPStemmer # para fazer a estemização em documentos da lingua portuguesa

import unicodedata # sera usada para remover acentos dos documentos em lingua portuguesa

# bibliotecas para geração de gráficos
import matplotlib.pyplot as plt

# biblioteca para tratar com caminho de arquivo
from pathlib import Path

# biblioteca para leitura de csv
import csv

# biblioteca para acessar diretórios e arquivos
import os

import re

print('Bibliotecas carregadas com sucesso')

Bibliotecas carregadas com sucesso


In [31]:
os.chdir(globals()['_dh'][0])

In [ ]:
nltk.download('all')

In [33]:
def ConverterMesParaNumero(texto):
    texto = texto.replace('janeiro', '1')
    texto = texto.replace('fevereiro', '2')
    texto = texto.replace('março', '3')
    texto = texto.replace('abril', '4')
    texto = texto.replace('maio', '5')
    texto = texto.replace('junho', '6')
    texto = texto.replace('julho', '7')
    texto = texto.replace('agosto', '8')
    texto = texto.replace('setembro', '9')
    texto = texto.replace('outubro', '10')
    texto = texto.replace('novembro', '11')
    texto = texto.replace('dezembro', '12')
    return texto

def ObterData(texto):
    data = re.findall(r'\d+', ConverterMesParaNumero(texto))[:3]
    if '-' in texto:
        data.reverse()
    
    return data

In [34]:
def ImportDatasetToArray(pathDataset):
    dataset = []
    
    with open(pathDataset, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        next(reader, None)
        
        for row in reader:
            dataset.append(row)
            
    return dataset

In [35]:
def ImportDatasetToArrayFakeBr(dirDataset, dirMetaDataset, tipo):
    dataset = []
    
    os.chdir(dirDataset)
    for i, file in zip(range(len(os.listdir())), os.listdir()):
        row = []
        file = open(file, 'r', encoding='utf-8-sig')
        row.append(i)
        row.append(file.read())
        dataset.append(row)
        
    os.chdir(dirMetaDataset)
    for i, file in zip(range(len(os.listdir())), os.listdir()):
        file = open(file, 'r', encoding='utf-8-sig')
        lines = file.readlines()
        data = ObterData(lines[3])
        dataset[i] = dataset[i] + data + [lines[1]]
            
    return dataset

In [36]:
def preprocessing(text, stemming = False, stopwords = False):
    """
    Funcao usada para fazer o tratamento de textos da lingua portuguesa
    
    Parametros: 
        text: variavel do tipo string que contem o texto que devera ser tratado
        
        stemming: variavel do tipo booleana que indica se a estemizacao deve ser aplicada ou nao
        
        stopwords: variavel do tipo booleana que indica se as stopwords devem ser removidas ou nao
    """
    
    # Lower case
    text = text.lower()
    
    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text) 
    
    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)

    #normaliza o símbolo de dólar
    regex = re.compile('[$]+')
    text = re.sub(regex, "<MONEY>", text)    
    
    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')  
    text = re.sub(regex, " ", text)
    
    # normaliza os numeros 
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "<NUMBER>", text)
    
    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('portuguese')]
        text = " ".join( words )
    
    # aplica estemização
    if stemming: 
        stemmer_method = RSLPStemmer()  
        words = text.split() # separa o texto em palavras
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )
    
    # remove palavras de apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )
    
    return text

In [37]:
datasetTrue = ImportDatasetToArray('./input/data/true.csv')
datasetFake = ImportDatasetToArray('./input/data/fake.csv')

datasetTrue = np.asarray(datasetTrue)
datasetFake = np.asarray(datasetFake)

In [38]:
for i, msg in enumerate(datasetTrue):
    datasetTrue[i][1] = preprocessing(datasetTrue[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetTrue)-1:
        print(' %d de %d' %(i+1, len(datasetTrue)), end='\r')
        
for i, msg in enumerate(datasetFake):
    datasetFake[i][1] = preprocessing(datasetFake[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetFake)-1:
        print(' %d de %d' %(i+1, len(datasetFake)), end='\r')

In [39]:
import pandas as pd
dataset = pd.DataFrame(datasetTrue)
dataset.to_csv('./output/preprocessed/basetcc/true.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [40]:
import pandas as pd
dataset = pd.DataFrame(datasetFake)
dataset.to_csv('./output/preprocessed/basetcc/fake.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [41]:
truePath, trueMetaPath = os.path.abspath('./input/data/true'), os.path.abspath('./input/data/true-meta-information')
fakePath, fakeMetaPath = os.path.abspath('./input/data/fake'), os.path.abspath('./input/data/fake-meta-information')

In [42]:
datasetTrueBr = ImportDatasetToArrayFakeBr(truePath, trueMetaPath, 'true')
datasetFakeBr = ImportDatasetToArrayFakeBr(fakePath, fakeMetaPath, 'fake')

datasetTrueBr = np.asarray(datasetTrueBr)
datasetFakeBr = np.asarray(datasetFakeBr)

In [43]:
for i, msg in enumerate(datasetTrueBr):
    datasetTrueBr[i][1] = preprocessing(datasetTrueBr[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetTrueBr)-1:
        print(' %d de %d' %(i+1, len(datasetTrueBr)), end='\r')
        
for i, msg in enumerate(datasetFakeBr):
    datasetFakeBr[i][1] = preprocessing(datasetFakeBr[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetFake)-1:
        print(' %d de %d' %(i+1, len(datasetFakeBr)), end='\r')

In [44]:
os.chdir(globals()['_dh'][0])

In [45]:
import pandas as pd
dataset = pd.DataFrame(datasetTrueBr)
dataset.to_csv('./output/preprocessed/fakebr/true.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [46]:
import pandas as pd
dataset = pd.DataFrame(datasetFakeBr)
dataset.to_csv('./output/preprocessed/fakebr/fake.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [47]:
trueNormalized = os.path.abspath('./input/data/size_normalized_texts/true')
fakeNormalized = os.path.abspath('./input/data/size_normalized_texts/fake')

In [48]:
datasetTrueNormalized = ImportDatasetToArrayFakeBr(trueNormalized, trueMetaPath, 'true')
datasetFakeNormalized = ImportDatasetToArrayFakeBr(fakeNormalized, fakeMetaPath, 'fake')

datasetTrueNormalized = np.asarray(datasetTrueNormalized)
datasetFakeNormalized = np.asarray(datasetFakeNormalized)

In [49]:
for i, msg in enumerate(datasetTrueNormalized):
    datasetTrueNormalized[i][1] = preprocessing(datasetTrueNormalized[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetTrueNormalized)-1:
        print(' %d de %d' %(i+1, len(datasetTrueNormalized)), end='\r')
        
for i, msg in enumerate(datasetFakeNormalized):
    datasetFakeNormalized[i][1] = preprocessing(datasetFakeNormalized[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetFakeNormalized)-1:
        print(' %d de %d' %(i+1, len(datasetFakeNormalized)), end='\r')

In [50]:
os.chdir(globals()['_dh'][0])

In [51]:
import pandas as pd
dataset = pd.DataFrame(datasetTrueNormalized)
dataset.to_csv('./output/preprocessed/normfakebr/true.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [52]:
import pandas as pd
dataset = pd.DataFrame(datasetFakeNormalized)
dataset.to_csv('./output/preprocessed/normfakebr/fake.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [53]:
trueTruncado = os.path.abspath('./input/data/textos_truncados/true.csv')
fakeTruncado = os.path.abspath('./input/data/textos_truncados/fake.csv')

In [54]:
datasetTrueTruncado = ImportDatasetToArray(trueTruncado)
datasetFakeTruncado = ImportDatasetToArray(fakeTruncado)

datasetTrueTruncado = np.asarray(datasetTrueTruncado)
datasetFakeTruncado = np.asarray(datasetFakeTruncado)

In [55]:
for i, msg in enumerate(datasetTrueTruncado):
    datasetTrueTruncado[i][1] = preprocessing(datasetTrueTruncado[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetTrueTruncado)-1:
        print(' %d de %d' %(i+1, len(datasetTrueTruncado)), end='\r')
        
for i, msg in enumerate(datasetFakeTruncado):
    datasetFakeTruncado[i][1] = preprocessing(datasetFakeTruncado[i][1], stemming = True, stopwords = True)
    
    if i%50==0 or i==len(datasetFakeTruncado)-1:
        print(' %d de %d' %(i+1, len(datasetFakeTruncado)), end='\r')

In [56]:
os.chdir(globals()['_dh'][0])

In [57]:
import pandas as pd
dataset = pd.DataFrame(datasetTrueTruncado)
dataset.to_csv('./output/preprocessed/normtcc/true.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [58]:
import pandas as pd
dataset = pd.DataFrame(datasetFakeTruncado)
dataset.to_csv('./output/preprocessed/normtcc/fake.csv', sep=';', index = False, encoding = 'utf-8-sig')